# Exam project of group 26
### Group members:
- Baltazar Dydensborg
- Johan Kielgast Ladelund
- Laura Weile
- Simon Juul Hansen

### Research Question:
???

### All of the dependency imports

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
from tqdm import tqdm
import warnings


## Scraping betting data from Oddsportal.com

In [ ]:
# Example of URLs
newest_season_url = 'https://www.oddsportal.com/soccer/denmark/superliga/results/'
previous_season_url = lambda year_span: f'https://www.oddsportal.com/soccer/denmark/superliga-{year_span}/results/'
previous_season_url('2020-2021')
prev_url_example = 'https://www.oddsportal.com/soccer/denmark/superliga-2020-2021/results/'

In [ ]:
# requesting html data
page = requests.get('https://www.oddsportal.com/soccer/denmark/superliga-2020-2021/results/#/page/2/')
soup = BeautifulSoup(page.content, 'html.parser')
soup.find('body')

## FIRST PROBLEM WITH ODDSPORTAL.COM:
### The HTML data is being retrieved via a JavaScript Query, so we can't directly get it via scraping the website with 'requests'.
### We need to find a solution to getting the HTML data via the JS query, possible via Selenium

# www.football-data.co.uk

In [ ]:
df = pd.read_csv('https://www.football-data.co.uk/mmz4281/2021/E0.csv')

In [ ]:
df

In [ ]:
url_template = lambda year_span: f'https://www.football-data.co.uk/mmz4281/{year_span}/E0.csv'
dict_df = {}

first_years = [f'{x:02}' for x in range(0,21)]
second_years = [f'{x:02}' for x in range(1,22)]
final_years = [x+y for x,y in zip(first_years,second_years)]
final_years

for year in final_years:
    df_temp = pd.read_csv(url_template(year), nrows = 1)
    df_temp2 = pd.read_csv(url_template(year), usecols = df_temp.columns)
    dict_df[year] = df_temp2

In [ ]:
df_temp = pd.read_csv(f'https://www.football-data.co.uk/mmz4281/{final_years[3]}/E0.csv',nrows=1)
df_temp.columns 
df_0203 = pd.read_csv(f'https://www.football-data.co.uk/mmz4281/{final_years[3]}/E0.csv', usecols = df_temp.columns)


In [ ]:
res = requests.get('https://fb.oddsportal.com/ajax-sport-country-tournament-archive/1/4l3rT0aP/X0/1/0/1/')
res.content

# Elpriser fra Nordpool

In [ ]:
url = 'https://www.nordpoolgroup.com/historical-market-data/'
el_res = requests.get(url)
soup = BeautifulSoup(el_res.content, 'html.parser')
soup.find('a')

In [ ]:
df = pd.read_csv('https://www.nordpoolgroup.com/4a2916/globalassets/marketdata-excel-files/elspot-prices_2021_hourly_dkk.xls', sep='\t', skiprows=10)
df

In [ ]:
dls = "https://www.nordpoolgroup.com/4a2916/globalassets/marketdata-excel-files/elspot-prices_2021_hourly_dkk.xls"
resp = requests.get(dls)

with open('2021.xlsx', 'wb') as output:
    output.write(resp.content)
    output.close()

# Vejr data fra DMI API

In [ ]:
# Constants

metObsAPIKey = 'c4503ba1-28d4-45c5-850a-974e98bbb3e0'
climateDataAPIKEy = 'ac27b332-bde2-4138-a53e-f0ca82cf3667'
stationId = 6183

dmi_url= lambda API_KEY: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&parameterId=temp_dry&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'
dmi_url_id= lambda API_KEY, id: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&parameterId={id}&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'
dmi_url_allID= lambda API_KEY: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'


In [18]:
def dmi_weather_data2(apiKey, stat, start_date, end_date, stationId = '06183'):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId={stationId}&parameterId={id}&bbox=7,54,16,58&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06184', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            json_dmi = requests.get(temp_url).json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['observed'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['observed'])
            temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df

        
    

In [47]:
def TransformData(df):
    df = df
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop_duplicates(['parameterId', 'date','hour'])\
            .drop(['created', 'observed', 'stationId'], axis = 1)\
            .sort_values(by = ['date', 'hour'], ascending = [False, False])\
            .copy()
    
    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index()

    return df_new
    

In [ ]:
df = dmi_weather_data2(metObsAPIKey, 'temp_dry', '2016-01-01', '2021-08-01')
df

In [16]:
list_of_stats=['wind_min', 'wind_max', 'visibility', 'cloud_cover', 'cloud_height', 'temp_dry', 'wind_speed', 'wind_dir', 'pressure', 'humidity']

dict_df = dict()
for stat in tqdm(list_of_stats):
    dict_df[stat] = dmi_weather_data2(metObsAPIKey, stat, '2016-01-01', '2021-08-01')

<ipython-input-15-bbbe62e71713>:62: UserWarning: The requested stat wind_min isn't available for the default station for the daterange: 2016-01-01 - 2016-01-02
  warnings.warn(f"The requested stat {stat} isn't available for the default station for the daterange: {start_date} - {end_date}")


In [41]:
new_df = pd.concat(dict_df.values())

In [48]:
transform_df = TransformData(new_df)

In [49]:
transform_df

parameterId,date,hour,cloud_cover,cloud_height,humidity,pressure,temp_dry,visibility,wind_dir,wind_max,wind_speed
0,2016-01-01,1,90.0,150.0,97.0,1021.1,5.3,8000.0,191.0,6.6,5.9
1,2016-01-01,2,90.0,150.0,97.0,1021.2,5.3,7000.0,187.0,6.6,6.0
2,2016-01-01,3,100.0,150.0,96.0,1021.6,5.5,8000.0,210.0,7.1,6.1
3,2016-01-01,4,100.0,150.0,98.0,1021.3,5.5,6000.0,202.0,6.0,5.3
4,2016-01-01,5,100.0,80.0,97.0,1021.5,5.5,6000.0,238.0,8.1,4.1
...,...,...,...,...,...,...,...,...,...,...,...
48436,2021-07-31,22,NaN,NaN,80.0,999.3,17.3,50000.0,281.0,13.4,11.8
48437,2021-07-31,23,NaN,NaN,76.0,1000.0,17.1,50000.0,285.0,12.0,10.5
48438,2021-08-01,0,NaN,NaN,73.0,1000.5,16.9,50000.0,275.0,14.6,12.0
48439,2021-08-01,1,NaN,NaN,76.0,1000.5,16.6,50000.0,273.0,13.1,11.4
